In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from pubchempy import get_cids, get_properties
import os
import pickle
import lmdb

from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  
import warnings
warnings.filterwarnings(action='ignore')
from multiprocessing import Pool, cpu_count
import random
import sys
from sklearn.cluster import KMeans
from rdkit.Chem import rdMolTransforms
from rdkit.Chem.rdMolAlign import AlignMolConformers
import re
import json
import copy
import time
import threading

from rdkit.Chem.Descriptors import MolWt, MolLogP, HeavyAtomMolWt
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit import DataStructs

from rdkit.DataStructs import FingerprintSimilarity
from multiprocessing import Pool

import multiprocessing as mp

from rdkit.Chem import Descriptors, Crippen



In [2]:
def smi2_2Dcoords(smi):
    mol = Chem.MolFromSmiles(smi)
    mol = AllChem.AddHs(mol)
    AllChem.Compute2DCoords(mol)
    coordinates = mol.GetConformer().GetPositions().astype(np.float32)
    len(mol.GetAtoms()) == len(coordinates), "2D coordinates shape is not align with {}".format(smi)
    return coordinates


def smi2_3Dcoords(smi,cnt):
    mol = Chem.MolFromSmiles(smi)
    mol = AllChem.AddHs(mol)
    coordinate_list=[]
    for seed in range(cnt):
        try:
            res = AllChem.EmbedMolecule(mol, randomSeed=seed)  # will random generate conformer with seed equal to -1. else fixed random seed.
            if res == 0:
                try:
                    AllChem.MMFFOptimizeMolecule(mol)       # some conformer can not use MMFF optimize
                    coordinates = mol.GetConformer().GetPositions()
                except:                    
                    coordinates = smi2_2Dcoords(smi)            
                    
            elif res == -1:
                mol_tmp = Chem.MolFromSmiles(smi)
                AllChem.EmbedMolecule(mol_tmp, maxAttempts=10, randomSeed=seed)
                mol_tmp = AllChem.AddHs(mol_tmp, addCoords=True)
                try:
                    AllChem.MMFFOptimizeMolecule(mol_tmp)       # some conformer can not use MMFF optimize
                    coordinates = mol_tmp.GetConformer().GetPositions()
                except:
                    coordinates = smi2_2Dcoords(smi) 
        except:
            coordinates = smi2_2Dcoords(smi) 

        assert len(mol.GetAtoms()) == len(coordinates), "3D coordinates shape is not align with {}".format(smi)
        coordinate_list.append(coordinates.astype(np.float32))
    return coordinate_list


def inner_smi2coords(content):
    smi = content
    target = [1,0]
    cnt = 10 # conformer num,all==11, 10 3d + 1 2d

    mol = Chem.MolFromSmiles(smi)
    if len(mol.GetAtoms()) > 400:
        coordinate_list =  [smi2_2Dcoords(smi)] * (cnt+1)
    else:
        coordinate_list = smi2_3Dcoords(smi,cnt)
        coordinate_list.append(smi2_2Dcoords(smi).astype(np.float32))
    mol = AllChem.AddHs(mol)
    atoms = [atom.GetSymbol() for atom in mol.GetAtoms()]  # after add H 
    return pickle.dumps({'atoms': atoms, 
    'coordinates': coordinate_list, 
    'mol':mol,'smi': smi, 'target': target}, protocol=-1)


def smi2coords(content):
    try:
        return inner_smi2coords(content)
    except:
        return None


def write_lmdb(df, outpath='./temp/', nthreads=30):
    
    sz = len(df)   

    name = 'temp.lmdb'
    content_list = df
    os.makedirs(outpath, exist_ok=True)
    output_name = os.path.join(outpath, name)
    try:
        os.remove(output_name)
    except:
        pass
    env_new = lmdb.open(
        output_name,
        subdir=False,
        readonly=False,
        lock=False,
        readahead=False,
        meminit=False,
        max_readers=1,
        map_size=int(100e9),
    )
    txn_write = env_new.begin(write=True)
    with Pool(nthreads) as pool:
        i = 0
        for inner_output in pool.imap(smi2coords, content_list):
            if inner_output is not None:
                txn_write.put(f'{i}'.encode("ascii"), inner_output)
                i += 1
        txn_write.commit()
        env_new.close()


In [3]:
def predict():
    try:
        os.remove('./temp/._temp.out.pkl')
    except:
        pass
    !python unimol/infer.py --user-dir ./unimol "temp" --task-name "" --valid-subset temp \
       --results-path "./temp" \
       --num-workers 30 --ddp-backend=c10d --batch-size 16 \
       --task mol_finetune --loss 'multi_task_BCE' --arch unimol_base \
       --classification-head-name "classification" --num-classes 2 \
       --dict-name 'dict.txt' --conf-size 11 \
       --only-polar 0  \
       --path "./bestmodel.pt"  \
       --fp16 --fp16-init-scale 4 --fp16-scale-window 256 \
       --log-interval 50 --log-format simple --no-shuffle

In [4]:
def get_results(predict_path, csv_path):
    predict = pd.read_pickle(predict_path)
    smi_list, predict_list1,predict_list2,target_list = [], [] , [],[]
    for batch in predict:
        sz = batch["bsz"]
        c = 0
        for i in range(sz):
            smi_list.append(batch["smi_name"][i])
            predict_list1.append(batch["prob"][i][0].cpu().tolist())
            predict_list2.append(batch["prob"][i][1].cpu().tolist())
            target = 1
            if batch["target"][c].cpu().tolist() == 0:
                target = 0

            target_list.append(target)
            c+=2

    predict_df = pd.DataFrame({"SMILES": smi_list, "predict_prob1": predict_list1,"predict_prob2": predict_list2, "target":target_list})
    predict_df = predict_df.groupby(["SMILES","target"])["predict_prob1","predict_prob2"].mean().reset_index()

    toxic = 0
    ntoxic = 0
    nontoxic = 0
    nnontoxic = 0
    toxiclist = []
    nontoxiclist = []
    for index, row in predict_df.iterrows():
        if row['predict_prob1'] > row['predict_prob2']:
            predict = 1
        else:
            predict = 0
        
        target = row['target']

        if predict == target:
            if predict ==1:
                toxic+=1
                nontoxiclist.append(row['SMILES'])
            else:
                nontoxic+=1
        else:
            if target == 1:
                ntoxic+=1
                toxiclist.append(row['SMILES'])

            else:
                nnontoxic+=1
    
    
    return toxiclist, nontoxiclist

In [5]:

toxicall = []
nontoxicall = []
def toxic_remover(sim_props):
    global toxicall, nontoxicall
    for i in toxicall:
        sim_props = sim_props[sim_props.SMILES != i]

    temp = list(set(sim_props['SMILES'].to_list()))


    gnnlist = [x for x in temp if x not in nontoxicall]

    try:
        os.remove('./temp/._temp.out.pkl')
    except:
        pass
    try:
        
        if gnnlist:
                                    
            write_lmdb(gnnlist)
            predict()
            predict_path='./temp/._temp.out.pkl'  
            csv_path='./temppredicted.csv'
            toxic, nontoxic = get_results(predict_path, csv_path)
            toxicall = toxicall + toxic
            nontoxicall = nontoxicall + nontoxic

            for i in toxic:
                sim_props = sim_props[sim_props.SMILES != i]

    except Exception as e:
        print("ERROR GNN")
        print("An error occurred:", e)                
        pass
    return sim_props


In [6]:

mol2 = Chem.MolFromSmiles('CCCCCCC(C)CCCCCCCCCOS(=O)(=O)O')
fp2 = MACCSkeys.GenMACCSKeys(mol2)

def calculate_similarity_reference(mol, fp2 = fp2, threshold = 0.9):
    """
    Calculate the similarity between two molecules using MACCS keys.
    
    :param smiles1: SMILES notation of the first molecule.
    :param smiles2: SMILES notation of the second molecule.
    :return: Tanimoto similarity coefficient.
    """
    # Convert the SMILES strings to RDKit molecule objects

    #mol2 = Chem.MolFromSmiles(smiles2)

    if mol is None or mol2 is None:
        raise ValueError("Invalid SMILES string provided.")

    # Generate the MACCS fingerprints
    fp1 = MACCSkeys.GenMACCSKeys(mol)
    #fp2 = MACCSkeys.GenMACCSKeys(mol2)

    # Calculate Tanimoto similarity
    similarity = DataStructs.TanimotoSimilarity(fp1, fp2)
    similarity = abs(similarity-threshold)
    return round(similarity, 2)

In [7]:
def calculate_similarity_for_smiles(target_fp, smiles, similarity_threshold):
    # Convert the SMILES to a RDKit molecule
    mol = Chem.MolFromSmiles(smiles)

    # If the molecule is None, return None to indicate an invalid SMILES
    if mol is None:
        return None

    # Compute the MACCS fingerprint for the molecule
    fp = MACCSkeys.GenMACCSKeys(mol)

    # Calculate the Tanimoto similarity
    similarity = FingerprintSimilarity(target_fp, fp)

    # Return the SMILES string if the similarity is above the threshold
    return smiles if similarity >= similarity_threshold else None

def calculate_similarity_parallel(search_space_df, target_smiles, similarity_threshold=0.9, num_workers=32):


    # Convert the target molecule to a RDKit molecule and compute its MACCS fingerprint
    target_mol = Chem.MolFromSmiles(target_smiles)
    if target_mol is None:
        raise ValueError("Invalid target SMILES string")
    target_fp = MACCSkeys.GenMACCSKeys(target_mol)

    # Use multiprocessing pool to parallelize the computation
    with Pool(num_workers) as pool:
        results = pool.starmap(calculate_similarity_for_smiles, [(target_fp, smiles, similarity_threshold) for smiles in search_space_df['SMILES']])

    # Filter out None and collect valid SMILES strings
    similar_smiles = [smiles for smiles in results if smiles is not None]

    # Create a DataFrame with similar molecules
    similar_df = pd.DataFrame(similar_smiles, columns=['SMILES'])

    return similar_df

In [8]:
def calculate_properties(smiles):
    """
    Calculate the properties (XlogP, Molecular Weight, Molecular Complexity) for a given SMILES string.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return smiles, None, None, None
    
    xlogp = round(Crippen.MolLogP(mol), 1)
    mol_wt = round(Descriptors.MolWt(mol), 1)
    mol_complexity = round(Descriptors.BertzCT(mol), 1)
    ref_likeness = round(calculate_similarity_reference(mol), 2)
    return smiles, xlogp, mol_wt, mol_complexity, ref_likeness

In [9]:
global_folder_path = ""
global_compound = ""
gen_counter = 0

In [10]:

search_space_df = pd.read_csv("SearchSpace.csv")
def build_space_from_parents(Parent_smiles, T_hat, lambda_):
    all_offsprings = pd.DataFrame()
    parent_counter = 1
    for Parent_smile in Parent_smiles['SMILES']:
        offspring_smiles = calculate_similarity_parallel(search_space_df, Parent_smile, similarity_threshold=T_hat, num_workers=32)

        if len(offspring_smiles) > lambda_:
            selected_offspring_smiles = offspring_smiles.sample(n=lambda_, replace=False, random_state=1)
        else:
            selected_offspring_smiles = offspring_smiles
        selected_offspring_smiles.to_csv(global_folder_path+"/Progression/_"+str(gen_counter)+"_"+str(parent_counter)+".csv",index=False)
        parent_counter+=1
        all_offsprings = pd.concat([all_offsprings, selected_offspring_smiles], ignore_index=True)

    if len(all_offsprings) > 0:
        all_offsprings = all_offsprings.drop_duplicates()
        
        all_offsprings = toxic_remover(all_offsprings)

        pool = mp.Pool(mp.cpu_count())

        # Processing the SMILES strings in parallel
        results = pool.map(calculate_properties, all_offsprings['SMILES'])

        # Converting results to a DataFrame
        all_offsprings = pd.DataFrame(results, columns=['SMILES', 'XLogP', 'MolecularWeight', 'Complexity', 'RefLikeness'])
        
        return all_offsprings , True
    else:
        return all_offsprings, False
    


In [11]:
def build_frontier_from_offsprings(all_offsprings, criteria):
    try:
        frontier = all_offsprings.iloc[[0]]  # initialises frontier with first compound in sim_props
    except:
        print(all_offsprings)
        print("NO OFFSPRINGS")
        sys.exit()
    for i in range(1, all_offsprings.shape[0]): # sweep sim_props: for each compound i in sim_props...
        s = ((all_offsprings.iloc[i][criteria.iloc[:,0]].values >= frontier[criteria.iloc[:,0]].values) & 
             (criteria.iloc[:,1].values == 'max')) | ((all_offsprings.iloc[i][criteria.iloc[:,0]].values <= frontier[criteria.iloc[:,0]].values) & 
                                                       (criteria.iloc[:,1].values == 'min'))
        domij = np.all(s, axis=1)
        domedij = np.any(s, axis=1)        
        if domij.any():
            frontier = frontier.drop(index=frontier[domij].index)  # remove dominated compounds
            frontier = pd.concat([frontier, all_offsprings.iloc[[i]]], ignore_index=True)  # add new compound
        elif domedij.any():
            if any(all_offsprings.iloc[[i]][col].isin(frontier[col]).any() for col in frontier.columns):
                s = ((all_offsprings.iloc[i][criteria.iloc[:,0]].values <= frontier[criteria.iloc[:,0]].values) & 
                    (criteria.iloc[:,1].values == 'max')) | ((all_offsprings.iloc[i][criteria.iloc[:,0]].values >= frontier[criteria.iloc[:,0]].values) & 
                                                       (criteria.iloc[:,1].values == 'min'))
                domedij = np.all(s, axis=1)
                if domedij.any():
                    continue                
            frontier = pd.concat([frontier, all_offsprings.iloc[[i]]], ignore_index=True)   
    return frontier.drop_duplicates()

In [12]:
def compare_lists(frontier_all, checker, criteria):
    # Step 1: Convert the 'SMILES' column into a list
    temp = build_frontier_from_offsprings(frontier_all,criteria)
    smiles_list = temp['SMILES'].tolist()

    # Step 2: Remove duplicates
    unique_smiles = list(set(smiles_list))

    # Step 3: Compare with 'checker' list
    return set(unique_smiles) == set(checker), unique_smiles

In [13]:
def algorithm(compound, criteria, T_hat, lambda_, G_hat, approx_factor,beta_,folder_path):
    global gen_counter
    templambda_ = lambda_
    tempbeta_ = beta_
    dftemp = pd.DataFrame({'SMILES': [compound]})
    print(compound)
    offsprings, flag = build_space_from_parents(dftemp, T_hat, lambda_) # initial compounds retrieved given "seed" compound
    if flag:
        
        frontier = build_frontier_from_offsprings(offsprings, criteria) # initial frontier 
    else:
        print(offsprings)
        print("NO FRONTIER")
        sys.exit()
        
    card_f_1 = frontier.shape[0] # number of compounds in initial frontier
    S_count = 1 # control variable to indicate in which generation we are

    start_time = time.time()
    end_time = start_time + 0.5 * 3600 / 5

    lambda_list = []
    beta_list = []

    frontier_all = frontier.copy()

    while time.time() < end_time:
        
        gen_counter +=1
        frontier_gen = pd.DataFrame(columns=frontier.columns)

        if len(frontier) > beta_:
            parents = frontier.sample(n=beta_, replace=False, random_state=1)
        else:
            parents = frontier

        if len(parents) == 0:
            break


        offsprings, flag = build_space_from_parents(parents, T_hat, lambda_)

        if not flag:
            break

        offsprings['Complexity'] = offsprings['Complexity'].astype(float)
        offsprings['MolecularWeight'] = offsprings['MolecularWeight'].astype(float)
        offsprings['RefLikeness'] = offsprings['RefLikeness'].astype(float)
        offsprings['XLogP'] = offsprings['XLogP'].astype(float)

        offsprings = offsprings[offsprings['Complexity'] <= 500]
        offsprings = offsprings[offsprings['MolecularWeight'] <= 500]
        #offsprings = offsprings[offsprings['RefLikeness'] <= 0.9]
        offsprings = offsprings[offsprings['XLogP'] >= 4]
        
        frontier_gen = build_frontier_from_offsprings(offsprings, criteria)

        card_f_2 = frontier_gen.shape[0] # number of compounds in "big frontier"

        frontier_all = pd.concat([frontier_all, frontier_gen], ignore_index=True)
        frontier = frontier_gen


        if (max(card_f_1, card_f_2) / min(card_f_1, card_f_2)) <= approx_factor: # if, given approximation, size of frontier is stable, then STOP
            pass
        else:
            if (card_f_2 / card_f_1) > 1: # if size of frontier is growing, then reduce lambda and build next generation
                lambda_ = round(lambda_ / G_hat)
                beta_ = round(beta_/ G_hat)
            if (card_f_2 / card_f_1) < 1: # if size of frontier is shrinking, then increase lambda and build next generation
                lambda_ = round(lambda_ * G_hat)
                beta_ = round(beta_* G_hat)
            card_f_1 = card_f_2
        
        S_count += 1

        lambda_list.append([S_count,lambda_])      
        beta_list.append([S_count,beta_])

        if S_count == 11:
            break
        
        if S_count == 100:

            frontier_all = frontier_all[frontier_all['Complexity'] <= 400]
            frontier_all = frontier_all[frontier_all['MolecularWeight'] <= 400]
            frontier_all = frontier_all[frontier_all['XLogP'] <= 10]

            frontier_all = frontier_all[frontier_all['Complexity'] >= 200]
            frontier_all = frontier_all[frontier_all['MolecularWeight'] >= 200]
            frontier_all = frontier_all[frontier_all['XLogP'] >= 5]

            frontier_all = build_frontier_from_offsprings(frontier_all, criteria)
            checker = frontier_all['SMILES'].tolist()
            
        elif S_count > 100 and S_count % 5 == 0:

            frontier_all = frontier_all[frontier_all['Complexity'] <= 400]
            frontier_all = frontier_all[frontier_all['MolecularWeight'] <= 400]
            frontier_all = frontier_all[frontier_all['XLogP'] <= 10]

            frontier_all = frontier_all[frontier_all['Complexity'] >= 200]
            frontier_all = frontier_all[frontier_all['MolecularWeight'] >= 200]
            frontier_all = frontier_all[frontier_all['XLogP'] >= 5]

            #frontier_all = build_frontier_from_offsprings(frontier_all,criteria)
            #smiles_list = frontier_all['SMILES'].tolist()
            flag, checker2 = compare_lists(frontier_all, checker, criteria)
            
            if flag:
                break

            checker = checker2

    
    frontier_all.to_csv(global_folder_path+global_compound+"_"+str(T_hat)+"_"+str(templambda_)+"_"+str(tempbeta_)+"_"+str(gen_counter)+"_Allfrontiers.csv",index=False)
    frontier_all['XLogP'] = frontier_all['XLogP'].astype(float)
    frontier_all['Complexity'] = frontier_all['Complexity'].astype(float)
    frontier_all['MolecularWeight'] = frontier_all['MolecularWeight'].astype(float)
    frontier_all['RefLikeness'] = frontier_all['RefLikeness'].astype(float)

    frontier_all = frontier_all[frontier_all['Complexity'] <= 400]
    frontier_all = frontier_all[frontier_all['MolecularWeight'] <= 400]
    frontier_all = frontier_all[frontier_all['XLogP'] <= 10]

    frontier_all = frontier_all[frontier_all['Complexity'] >= 200]
    frontier_all = frontier_all[frontier_all['MolecularWeight'] >= 200]
    frontier_all = frontier_all[frontier_all['XLogP'] >= 5]

    #frontier = build_frontier_from_offsprings(frontier_all, criteria)

    return frontier_all, S_count, lambda_list, beta_list, time.time() - start_time

In [14]:
criteria = pd.DataFrame(columns=['variable_name', 'rule'])
criteria.loc[0] = ['XLogP', 'max']
criteria.loc[1] = ['Complexity', 'min']
criteria.loc[2] = ['MolecularWeight', 'min']
criteria.loc[3] = ['RefLikeness', 'min']

In [15]:
detergent = ["CCCCCCC(C)CCCCCCCCCOS(=O)(=O)O",
             "CCCCCCCCC(C)CCCCCCCOS(=O)(=O)O",
             "CCCCCCCCC(CCCCCC)CCOS(=O)(=O)O",
             "C(C(=O)O)C(CC(=O)O)(C(=O)O)O.[Na]",
             "CCCCCCCCC(C)CCCCCCCO.OS(=O)(=O)O",
             "CCCCCCC(C)CCCCCCCCCO.OS(=O)(=O)O",
             "CCCCCCCCC(CCCCCC)CCO.OS(=O)(=O)O",
             "CCCCCCCCC(=C)CCCCCC",
             "CCCCCCC(C)CCCCCCCCCO",
             "CCCCCCCCC(C)CCCCCCCO",
             "CCCCCCCCC(CCCCCC)CCO",
             "CCOOS(=O)(=O)O",
             "C(CO)N(CCO)CCO",
             "[AsH3]"]

In [16]:
reference_smiles_around = [
'CCCCCCCCCCCCCOCCC(CC(=O)[O-])(C(=O)[O-])S(=O)(=O)O',
'CC(C)CCCCCCCCCCCCCCCC(C(C(C(C(CO)OS(=O)(=O)O)O)O)O)O',
'CCCCCCCCOS(=O)(=O)[O-].CCCCCCCCOS(=O)(=O)[O-].[Cu+2]',    
'CCCCC(CC)COC(=O)C(C)S(=O)(=O)O',
'CCC=CCC=CCC=CCCCCCCC(C)C(=O)S(=O)(=O)O'
]

In [17]:

def start_around_detergent(reference_smiles_around,criteria,T_hat, lambda_, G_hat, approx_factor,beta_):
    global global_folder_path, global_compound
    f = pd.DataFrame()
    for compound in reference_smiles_around:
        
        folder_path = './resultscmaes_around/'+"CCCCCCC(C)CCCCCCCCCOS(=O)(=O)O"+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        folder_path = './resultscmaes_around/'+"CCCCCCC(C)CCCCCCCCCOS(=O)(=O)O"+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_) +"/"

        if not os.path.exists(folder_path+"/Progression"):
            os.makedirs(folder_path+"/Progression")
        

        global_compound = "CCCCCCC(C)CCCCCCCCCOS(=O)(=O)O"
        global_folder_path = folder_path
        templambda_ = lambda_
        tempbeta_ = beta_
        ftemp, S_count, lambda_list, beta_list, time = algorithm(compound, criteria, T_hat, templambda_, G_hat, approx_factor,tempbeta_,folder_path)
        ftemp = ftemp[['SMILES','Complexity','MolecularWeight','XLogP','RefLikeness']]
        f = pd.concat([f, ftemp], ignore_index=True)

    try:
        f1 = build_frontier_from_offsprings(f, criteria)
    except:
        pass
    
    f1.to_csv(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_final.csv",index=False)


    f = f[f['Complexity'] <= 350]
    f = f[f['MolecularWeight'] <= 350]
    f = f[f['XLogP'] <= 10]

    f = f[f['Complexity'] >= 250]
    f = f[f['MolecularWeight'] >= 250]
    f = f[f['XLogP'] >= 5]

    try:
        f = build_frontier_from_offsprings(f, criteria)
    except:
        pass
    
    f.to_csv(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_cleaned_final.csv",index=False)


    with open(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_toxic.txt", 'w') as file:
        for item in toxicall:
            file.write(f"{item}\n")
    
    with open(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_nontoxic.txt", 'w') as file:
        for item in nontoxicall:
            file.write(f"{item}\n")
    
    with open(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_lambdachange.txt", 'w') as file:
        for item in lambda_list:
            file.write(f"{item}\n")
    
    with open(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_betachange.txt", 'w') as file:
        for item in beta_list:
            file.write(f"{item}\n")

    with open(folder_path+compound+"_"+str(T_hat)+"_"+str(lambda_)+"_"+str(beta_)+"_runtime.txt", 'w') as file:
        file.write(f"{time}\n")
        file.write(f"{S_count}\n")

In [18]:
lambda_runlist = [50]                          # (5-15) 11 or 6    | 3     5,10,15 
T_hat_runlist =  [0.9]                        # (0.93-1) 8 or 5   | 3     95,97,99  
beta_runlist = [10]
G_hat = 1.25                                # CONSTANT
approx_factor = 1.1                         # CONSTANT

In [19]:
for T_hat in T_hat_runlist:
    for lambda_ in lambda_runlist:
        for beta_ in beta_runlist:
            toxicall = []
            nontoxicall = []
            start_around_detergent(reference_smiles_around,criteria,T_hat, lambda_, G_hat, approx_factor,beta_)

CCCCCCCCCCCCCOCCC(CC(=O)[O-])(C(=O)[O-])S(=O)(=O)O


2024-01-17 17:41:47 | INFO | unimol.inference | loading model(s) from ./bestmodel.pt
2024-01-17 17:41:47 | INFO | unimol.tasks.unimol_finetune | dictionary: 30 types
2024-01-17 17:41:49 | INFO | unimol.inference | Namespace(no_progress_bar=False, log_interval=50, log_format='simple', tensorboard_logdir='', seed=1, cpu=False, fp16=True, bf16=False, bf16_sr=False, allreduce_fp32_grad=False, fp16_no_flatten_grads=False, fp16_init_scale=4, fp16_scale_window=256, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir='./unimol', empty_cache_freq=0, all_gather_list_size=16384, suppress_crashes=False, profile=False, ema_decay=-1.0, loss='multi_task_BCE', optimizer='adam', lr_scheduler='fixed', task='mol_finetune', num_workers=30, skip_invalid_size_inputs_valid_test=False, batch_size=16, required_batch_size_multiple=8, data_buffer_size=10, train_subset='train', valid_subset='temp', validate_interval=1, validate_interval_updates=0, validate_after_updates=0, fixed_v